In [ ]:
import tensorflow as tf 
import keras as ks 
import numpy as np
import script_config as sc


# Calculated for 2 Hop
maximum_neighborhood_size = sc._config_maximum_neighborhood_size
data_folder               = sc._config_data_folder
random_seed               = sc._config_random_seed
availible_threads         = sc._config_availible_threads
feature_permutations      = sc._config_feature_permutations

In [ ]:
def create_undirected_adjacency(filename):
    
    loaded_data = np.load(filename)
    users       = loaded_data['local_neighbors']
    relations   = loaded_data['local_relations'] 
    adj         = np.zeros((len(users),len(users)))
    
    # Randomly Permute all nodes except the first one 
    users[1:] = np.random.permutation(users[1:])
    
    for relation in relations: 
        idx_1 = np.where(users[:,0] is relations[2]) 
        idx_2 = np.where(users[:,0] is relations[3])
        adj[idx_1,idx_2] = 1
        adj[idx_2,idx_1] = 1
    
    return (tf.convert_to_tensor(adj),tf.constant(users[0,0]))

def create_spectral_decomposition(adjacency): 
    
    n_nodes              = tf.shape(adjacency)[0]
    
    degrees_inv_sqrt     = tf.reshape(tf.math.pow(tf.math.sqrt(tf.math.reduce_sum(adjacency,axis=0)), 
                                                  tf.constant([-1]*n_nodes)),[n_nodes,1]) 
    
    normalized_laplacian = tf.subtract(tf.eye(n_nodes),
                                       tf.multiply(adjacency,
                                                   tf.linalg.matmul(degrees_inv_sqrt, 
                                                                    tf.transpose(degrees_inv_sqrt)) ))

    e_values, e_vectors   = tf.linalg.eigh(normalized_laplacian)
    
    features              = tf.multiply(e_values,e_vectors[0,:])
    
    return features 
    

In [ ]:
!mkdir /nvme/drive_1/NTDS_Final/spectral_decomp 
!rm -r /nvme/drive_1/NTDS_Final/spectral_decomp/filtered
!mkdir /nvme/drive_1/NTDS_Final/spectral_decomp/filtered

In [ ]:
np.random.seed(seed=random_seed)

# Start with tensor of files containing adjacency lists 
file_name_tensor = tf.data.Dataset.list_files("/nvme/drive_1/NTDS_Final/local_list/filtered/*.csv",
                                              shuffle=True,seed=random_seed)

# Create adjacency matrices and interleave versions with permuted node indices  
adjacency_tensor, ground_truth = file_name_tensor.interleave(lambda x:
                                       tf.data.TextLineDataset(x)
                                        .repeat(feature_permutations)
                                           .map(create_undirected_adjacency, 
                                                num_parallel_calls=tf.constant(availible_threads)),
                       cycle_length=tf.constant(feature_permutations), 
                       block_length=1, 
                       num_parallel_calls=tf.constant(availible_threads))

#
feature_tensor = adjacency_tensor.shuffle(tf.constant(100),seed=random_seed)\
                                 .map(create_spectral_decomposition,num_parallel_calls=tf.constant(availible_threads))

In [ ]:
model = ks.Sequential([
            ks.layers.Dense(128, activation=tf.nn.relu),
            ks.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])